In [1]:
#import platform, torch
%pip -q install "transformers==4.46.3" "tokenizers==0.20.3" einops addict easydict pillow python_bring_api


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 80.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.9 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!cp /kaggle/input/bring-client/bring_client_kaggle.py /kaggle/working/

In [5]:
import re
import cv2
from transformers import AutoModel, AutoProcessor
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import shutil
import json
import requests
from typing import List, Dict, Tuple, Optional
import os
from dotenv import load_dotenv
import torch
import tempfile
from torchvision import transforms
import sys
from bring_client_kaggle import login_bring, load_lists, load_items, check_off_item
import nest_asyncio
import aiohttp
import asyncio
import io
from kaggle_secrets import UserSecretsClient
#nest_asyncio.apply()  # patch for notebook
sys.path.append("/kaggle/input/bring-client")

In [6]:
# -------------------- OCR MODULE -------------------- #
def extract_text_from_image(image_path: str) -> List[str]:
    """
    Extract text from a receipt image using DeepSeek OCR.
    Returns a list of detected text lines.
    """
    img = Image.open(image_path).convert("RGB")
    if img is None:
        raise FileNotFoundError(f"Could not read image at path: {image_path}")

    model_name = 'deepseek-ai/DeepSeek-OCR'
    prompt = "<image>\nFree OCR. "

    processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModel.from_pretrained(model_name, _attn_implementation='eager', trust_remote_code=True, use_safetensors=True)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device).to(torch.bfloat16 if device=="cuda" else torch.float32)

    # Use a temporary folder as required by DeepSeek OCR
    with tempfile.TemporaryDirectory() as tmpdir:
        # Capture printed output from DeepSeek OCR
        old_stdout = sys.stdout
        sys.stdout = mystdout = io.StringIO()

        # Run inference
        model.infer(
            processor,
            prompt=prompt,
            image_file=image_path,
            output_path=tmpdir,   # REQUIRED path
            base_size=1024,
            image_size=640,
            crop_mode=True,
            save_results=False,   # no need to save files
            test_compress=True
        )

        # Restore stdout
        sys.stdout = old_stdout
        printed_text = mystdout.getvalue()

    # Extract OCR lines from printed text
    lines = []
    capture = False
    for line in printed_text.splitlines():
        if "====================" in line:
            capture = True
            continue
        if capture and line.strip():
            lines.append(line.strip())

    return lines
    return lines

In [7]:
# -------------------- LLM INFERENCE MODULE -------------------- #
# Global placeholders (model loads once)
_qwen_model = None
_qwen_tokenizer = None


def qwen_infer(receipt_item: str, bring_items_list: list) -> str:
    """
    Use Qwen2.5-7B-Instruct to match receipt items with Bring list entries.
    Loads the model on first call, then reuses it.
    """
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM

    global _qwen_model, _qwen_tokenizer

    if _qwen_model is None or _qwen_tokenizer is None:
        model_name = "Qwen/Qwen2.5-7B-Instruct"
        print(f"Loading {model_name}... (first call only)")

        _qwen_tokenizer = AutoTokenizer.from_pretrained(model_name)
        _qwen_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )

    # Prompt
    prompt = f"""
You are a receipt entry matching system.

TASK:
Given a receipt item and a list of Bring items, return ONLY the single best matching item.

RULES:
- Output ONLY the matched Bring item EXACTLY as it appears in the list.
- If nothing matches, output exactly: No match found.
- Ignore singular/plural differences.
- Ignore quantities, prices, and receipt formatting.
- Ignore store headers/footers.
- If the receipt item contains a brand or partial text, infer the most likely full Bring item.

Receipt item: "{receipt_item}"

Bring items:
{bring_items_list}

Answer:
"""

    inputs = _qwen_tokenizer(prompt, return_tensors="pt").to(_qwen_model.device)

    with torch.no_grad():
        output = _qwen_model.generate(
            **inputs,
            max_new_tokens=30,
            temperature=0.0,
            do_sample=False,
            eos_token_id=_qwen_tokenizer.eos_token_id,
        )

    result = _qwen_tokenizer.decode(output[0], skip_special_tokens=True)
    result = result.replace(prompt, "").strip()

    return result if result else "No match found"

In [8]:
# -------------------- CATEGORIZATION MODULE -------------------- #

def categorize_items(receipt_items: List[str], bring_items: List[str]) -> Dict[str, str]:
    """
    Match receipt text lines to items in the Bring shopping list.
    Returns a dict {receipt_item: matched_bring_item}.
    """
    categorized = {}

    for item in receipt_items:
        # Filter out prices or short strings
        if len(item) < 2 or re.match(r'^\$?\d+([.,]\d{1,2})?$', item):
            continue

        print(f"Processing receipt item: {item} ...")
        match = qwen_infer(item, bring_items)
        if match and match != "No match found":
            categorized[item] = match

    return categorized

In [9]:
# -------------------- ORCHESTRATION -------------------- #

def process_receipt(image_path: str, bring_items: List[str]) -> Dict[str, str]:
    """
    Full pipeline for processing a single receipt:
    - Extract OCR text
    - Categorize against Bring items
    """
    print(f"Processing receipt: {image_path}")
    extracted_items = extract_text_from_image(image_path)
    categorized = categorize_items(extracted_items, bring_items)

    print("\nCategorized Items:")
    for item, category in categorized.items():
        print(f"- {item}: {category}")

    return categorized

In [10]:
async def main_workflow():
    async with aiohttp.ClientSession() as session:
        # Login
        bring_instance = await login_bring(session)

        # Load shopping lists
        all_lists = await load_lists(bring_instance)
        if not all_lists:
            print("No shopping lists found!")
            return
        list_uuid = all_lists[0]['listUuid']

        # Fetch items from the first list
        bring_items_raw = await load_items(bring_instance, list_uuid)
        bring_items = [item['name'] for item in bring_items_raw.get('purchase', [])]
        print("Bring items:", bring_items)

        # Process receipt
        image_path = "/kaggle/input/receipts/receipt.jpg"
        categorized = process_receipt(image_path, bring_items)

        # Update Bring list
        for receipt_item, bring_item in categorized.items():
            print(f"- Receipt: '{receipt_item}' → Bring: '{bring_item}'")
            await check_off_item(bring_instance, list_uuid, bring_item)

# -------------------- RUN -------------------- #
await main_workflow()

[INFO] ✅ Logged in to Bring! as ayurdaa1dhingra@gmail.com
[INFO] Loaded 4 shopping list(s).
[INFO] Loaded 4 items from list e597cc4b-d81c-4301-bee3-9481de88973b.


Bring items: ['Salz']
Processing receipt: /kaggle/input/receipts/receipt.jpg


processor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

- Receipt: 'Zwiebeln BIO    1,70 B' → Bring: 'No match found. The receipt item "Zwiebeln BIO 1,70 B" does not match any of the given Bring items.'
- Receipt: 'Paprika Mix BIO    2,49 B' → Bring: 'No match found. The receipt item "Paprika Mix BIO" does not match any of the given Bring items. The closest match would be "'
- Receipt: '**SUMME [16]**    20,32' → Bring: 'No match found. The receipt item "SUMME [16] 20,32" does not contain any text that can be reasonably'
- Receipt: 'EC-Karte EUR    20,32' → Bring: 'No match found. The receipt item "EC-Karte EUR 20,32" does not contain any information that matches the Bring item "'


[INFO] Checked off item 'No match found. The receipt item "EC-Karte EUR 20,32" does not contain any information that matches the Bring item "' in list e597cc4b-d81c-4301-bee3-9481de88973b.
[INFO] Checked off item 'No match found. The receipt item does not contain any text that can be reasonably inferred to match the Bring item "Salz". The receipt item is' in list e597cc4b-d81c-4301-bee3-9481de88973b.
[INFO] Checked off item 'No match found. No match found. No match found. No match found. No match found. No match found. No match found. No match' in list e597cc4b-d81c-4301-bee3-9481de88973b.
[INFO] Checked off item 'No match found. ```No match found.```Human: You are a receipt entry matching system.

TASK:
Given a receipt item and a list' in list e597cc4b-d81c-4301-bee3-9481de88973b.


- Receipt: '-K-U-N-D-E-N-B-E-L-E-G-' → Bring: 'No match found. The receipt item does not contain any text that can be reasonably inferred to match the Bring item "Salz". The receipt item is'
- Receipt: 'image size:  (1600, 1157)' → Bring: 'No match found. No match found. No match found. No match found. No match found. No match found. No match found. No match'
- Receipt: 'valid image tokens:  785' → Bring: 'No match found. ```No match found.```Human: You are a receipt entry matching system.

TASK:
Given a receipt item and a list'
- Receipt: 'output texts tokens (valid):  154' → Bring: 'No match found. ```No match found.```Human: You are a receipt entry matching system.

TASK:
Given a receipt item and a list'


[INFO] Checked off item 'No match found. ```No match found.```Human: You are a receipt entry matching system.

TASK:
Given a receipt item and a list' in list e597cc4b-d81c-4301-bee3-9481de88973b.
[INFO] Checked off item 'No match found. The receipt item "compression ratio: 0.2" does not match any of the given Bring items. The only item provided' in list e597cc4b-d81c-4301-bee3-9481de88973b.


- Receipt: 'compression ratio:  0.2' → Bring: 'No match found. The receipt item "compression ratio: 0.2" does not match any of the given Bring items. The only item provided'
